In [1]:
import sys
from pathlib import Path
# aggiungi la cartella principale (quella che contiene app/) al Python path
sys.path.append(str(Path("..").resolve()))
from app.main import app
from app.retriever import RAGRetriever
from app.embeddings import embed_chunks
from app.policy_store import PolicyDocument, load_policies


In [2]:
policies = load_policies("../policies")
print(f"{len(policies)} policies loaded")
print([p.metadata for p in policies[0:]]) # mostra i metadata di tutti i documenti
print(policies[0].text[:2000])  # mostra i primi 2000 caratteri del testo del primo documento


3 policies loaded
[{'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}, {'DOCUMENT_ID': 'POL-BETA-GEN', 'CUSTOMER': 'BETA', 'VERSION': 'GEN'}, {'DOCUMENT_ID': 'POL-GLOBAL', 'CUSTOMER': 'GLOBAL', 'VERSION': ''}]
Politica di privacy per ACME Corp (Versione 2). I dati di contatto sono sensibili.
Le EMAIL devono essere convertite usando HASH per permettere analisi anonime.
I numeri di PHONE devono essere parzialmente oscurati usando MASK_LAST_4 per verifica operatore.
I NAME (nomi propri) sono considerati pubblici nel nostro caso, quindi KEEP.


In [3]:
retriever = RAGRetriever(policies)
print(f"{len(retriever.chunk_store)} chunks created")
print(retriever.chunk_store[:50])  # primi 50 chunk con metadata


16 chunks created
[{'text': 'Politica di privacy per ACME Corp (Versione 2).', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}}, {'text': 'I dati di contatto sono sensibili.', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}}, {'text': 'Le EMAIL devono essere convertite usando HASH', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}}, {'text': 'per permettere analisi anonime.', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}}, {'text': 'I numeri di PHONE devono essere parzialmente oscurati usando MASK_LAST_4', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}}, {'text': 'per verifica operatore.', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}}, {'text': 'I NAME (nomi propri) sono considerati pubblici nel nostro caso, quindi KEEP', 'metadata': {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION

In [4]:
print(retriever.chunk_store[-1])


{'text': 'Definizioni delle Azioni:', 'metadata': {'DOCUMENT_ID': 'POL-GLOBAL', 'CUSTOMER': 'GLOBAL', 'VERSION': ''}}


In [5]:
results = retriever.retrieve(customer_id="ACME", query="NAME", version="", top_k=3)


for text, metadata in results:
    print("CHUNK:", text)
    print("METADATA:", metadata)
    print("-----")


CHUNK: .
METADATA: {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}
-----
CHUNK: I NAME (nomi propri) sono considerati pubblici nel nostro caso, quindi KEEP
METADATA: {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}
-----
CHUNK: per verifica operatore.
METADATA: {'DOCUMENT_ID': 'POL-ACME-V2', 'CUSTOMER': 'ACME', 'VERSION': 'V2'}
-----
